In [5]:
from agents.rag_agent import chat_agent

✅✅call agent step
get graph
get llm 


In [6]:
# from chains.semantic_search_chunk_chain import chunk_retriever
from chains.semantic_search_chunk_chain import get_chunk_retriever


In [7]:
import pandas as pd
df = pd.read_csv("testset_5files.csv")
question = df['user_input']
ground_truth = df['reference']
retrieval_context = df['reference_contexts']

In [12]:
get_chunk_retriever().invoke("ĐOÀN TNCS HỒ CHÍ MINH là gì?")[0].page_content

"content: BẠN CHẮP HAlẸ’iẾH’lỉ TRUNGƯƠNG ĐOÀN TNCS HỎ CHÍ MINH Số: 4244-QD/TWĐTN-TNTH Hà Nội, ngàylÿ tháng ÁQ năm 2019 - (2UYET ĐỊNH Về việc ban hành Quy chế giải thưởng “/Vhä giáo trẻ tiêu biễu? uốp Trung ương BẠN BÍ THƯ TRUNG ƯƠNG ĐOÀN TNCS HỖ CHÍ MINH - Căn cứ Điều 31, Chương IX, Điều lệ Đoàn TNCS Hỗ Chí Minh; - Căn cứ Quy chề hoạt íz'ọng của Ban Chấp hành 7ì Tung ương Đoàn TNCS Hồ Chí Minh khoá XI, Quy chế thì đua, .khen thưởng của Đoàn TNCS Hồ Chí Miình ban hành kèm theo Quyết định số 1 09-QĐ/TWĐTN-VP ngày 15/8/2018 của Ban Thường vụ Trung ương Đoàn TNCS Hồ Chí Minh; _ - Xét đề nghị của Ban Thanh niên Trường học Trung ương Đoàn, QUYẾT ĐỊNH Điều 1. Ban hành kèm theo Quyết định mty Quy chế Ẻai thưởng “Nhà giáo trẻ tiêu biểu” cấp Trung ương dành cho giáo viên, giảng viên trẻ đang công tác, giảng dạy trong các trường, cơ sở giáo , dục đào tạo, cơ sở giáo dục nghề nghiệp thuộc hệ thống giáo dục quoc dân. Điều 2. Ban Thanh niên Trường học Trung ương Đoàn là cơ quan thường - trực giải th

In [ ]:
answers = []
contexts = []
for query in question:
  answers.append(chat_agent.invoke(
    {"input": query},
    {"configurable": {"session_id": 1}}).get("output", "No output text provided."))
  contexts.append([docs.page_content for docs in get_chunk_retriever().invoke(query)])

In [14]:
df["response"] = answers
df["retrieved_contexts"] = contexts
df.to_csv("test_set_rag_13-12.csv")

In [31]:
import ast

data = {
    "user_input": question,
    "answer": answers,
    "contexts": contexts,
    "reference": df['reference'],
    "reference_contexts": df["reference_contexts"].apply(ast.literal_eval)}

In [40]:
from ragas.dataset_schema import EvaluationDataset
# from ragas import 
from datasets import Dataset
import pandas as pd
import ast
df = pd.read_csv('test_set_rag_13-12.csv')
df['retrieved_contexts'] = df['retrieved_contexts'].apply(ast.literal_eval)
df['reference_contexts'] = df['reference_contexts'].apply(ast.literal_eval)

dataset = Dataset.from_pandas(df)
# dataset= Dataset.from_dict(**data)

In [41]:
from ragas import evaluate
from ragas.cost import get_token_usage_for_openai

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    token_usage_parser=get_token_usage_for_openai,
)

df = result.to_pandas()

Evaluating: 100%|██████████| 60/60 [01:37<00:00,  1.62s/it]


In [43]:
print(result.total_tokens())


input_tokens=275559 output_tokens=38709 model=''


In [42]:
df.to_csv("final_result_13-12.csv")

<bound method EvaluationResult.total_tokens of {'context_precision': 0.9611, 'context_recall': 0.7844, 'faithfulness': 0.6798, 'answer_relevancy': 0.8966}>


In [43]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

In [44]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from llm.get_llm import get_eval_model_function, get_embedding_function
evaluator_llm = LangchainLLMWrapper(get_eval_model_function())
evaluator_embeddings = LangchainEmbeddingsWrapper(get_embedding_function())

get llm 
get_embedding_function


In [45]:
from ragas.cost import get_token_usage_for_openai

metrics = [
    LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
    
]
results = evaluate(dataset=dataset, metrics=metrics, token_usage_parser=get_token_usage_for_openai,)

Evaluating:  52%|█████▏    | 31/60 [02:58<03:43,  7.69s/it]Exception raised in Job[2]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Evaluating: 100%|██████████| 60/60 [05:58<00:00,  5.97s/it]


KeyError: 0

NameError: name 'result' is not defined